In [106]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xraylib
from math import sqrt
import torch
%matplotlib qt 

In [107]:
f = h5py.File('panagia1_mm.h5', 'r')
dset=f['dataset']
dEset=f['energies']
spectra_p=np.array(dset)
energies=np.array(dEset)
f.close()

f = h5py.File('christ.h5', 'r')
dset=f['dataset']
dEset=f['energies']
spectra_c=np.array(dset)
f.close()



In [108]:
f = h5py.File('spectra.h5', 'r')
dset=f['dataset']
spectra_h=np.array(dset)
f.close()

In [109]:
channels=100
rows_p=41
cols_p=65
rows_c=31
cols_c=46
rows_h=41
cols_h=21

In [110]:
spectra_p = spectra_p.astype(np.float32)
spectra_c = spectra_c.astype(np.float32)
spectra_h = spectra_h.astype(np.float32)
spectra_p=np.delete(spectra_p,np.s_[:channels],1)
spectra_c=np.delete(spectra_c,np.s_[:channels],1)
spectra_h=np.delete(spectra_h,np.s_[:channels],1)

energies=np.delete(energies,np.s_[:channels])

In [44]:
X_h.shape

torch.Size([861, 1948])

In [111]:
#roi function    
def fwhm(ev):
    print(sqrt(2.47*ev+4400))
    return sqrt(2.47*ev+4400)

In [112]:


X_p=torch.tensor(spectra_p)
X_c=torch.tensor(spectra_c)
X_h=torch.tensor(spectra_h)

In [113]:
#dic with energies of elements
dic={ 'Ca':3692,'Fe':6405,'Cu':8046,'Hg':9989,'Pb':10551,'Au':9713}
key_list=list(dic.keys())
min_ch=[]
max_ch=[]
min_roi_all=[]
max_roi_all=[]
for element in dic:
    min_roi=(dic[element]-fwhm(dic[element])/2)
    min_roi_all.append(min_roi)
    max_roi=(dic[element]+fwhm(dic[element])/2)
    max_roi_all.append(max_roi)
    min_ch.append(int((min_roi+960)/20)-channels)
    max_ch.append(int((max_roi+960)/20)-channels)
print(min_ch)
print(max_ch) 

116.27226668470861
116.27226668470861
142.1982770641051
142.1982770641051
155.7999358151344
155.7999358151344
170.50756581454092
170.50756581454092
174.53071362943544
174.53071362943544
168.49661717672555
168.49661717672555
[129, 264, 346, 443, 471, 429]
[135, 271, 354, 451, 479, 437]


In [114]:
spectra_h.shape

(861, 1948)

In [115]:
spectra_p_cube=spectra_p.reshape(rows_p,cols_p,spectra_p.shape[1])
spectra_c_cube=spectra_c.reshape(rows_c,cols_c,spectra_c.shape[1])
spectra_h_cube=spectra_h.reshape(cols_h,rows_h,spectra_h.shape[1])

In [125]:
%matplotlib qt
f1=plt.figure()
plt.imshow(np.sum(spectra_h_cube[:,:,min_ch[2]:max_ch[2]],axis=2))
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()
plt.rcParams.update({'font.size': 8})

In [116]:
y_p=np.zeros([len(spectra_p),6])
for i in range(len(spectra_p)):
    for j in range(len(key_list)):
        y_p[i][j]=np.sum(np.sum(spectra_p[i,min_ch[j]:max_ch[j]]))
y_c=np.zeros([len(spectra_c),6])
for i in range(len(spectra_c)):
    for j in range(len(key_list)):
        y_c[i][j]=np.sum(np.sum(spectra_c[i,min_ch[j]:max_ch[j]]))
y_h=np.zeros([len(spectra_h),6])
for i in range(len(spectra_h)):
    for j in range(len(key_list)):
        y_h[i][j]=np.sum(np.sum(spectra_h[i,min_ch[j]:max_ch[j]]))

In [73]:
y_p_cube=y_p.reshape(rows_p,cols_p,6)
y_c_cube=y_c.reshape(rows_c,cols_c,6)
y_h_cube=y_h.reshape(cols_h,rows_h,6)

In [117]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(hidden_layer_sizes=(100,),activation='relu',solver='adam',verbose=True,max_iter=300)



In [118]:
regr.fit(X_h, y_h)


Iteration 1, loss = 11636.95099561
Iteration 2, loss = 8819.06579037
Iteration 3, loss = 6775.11810196
Iteration 4, loss = 5106.44676427
Iteration 5, loss = 3824.97357580
Iteration 6, loss = 2734.36469494
Iteration 7, loss = 1854.37505846
Iteration 8, loss = 1169.99919444
Iteration 9, loss = 708.10143438
Iteration 10, loss = 437.41192396
Iteration 11, loss = 296.19371852
Iteration 12, loss = 224.84236647
Iteration 13, loss = 182.28609741
Iteration 14, loss = 145.54718723
Iteration 15, loss = 113.73966563
Iteration 16, loss = 95.53986779
Iteration 17, loss = 86.97277144
Iteration 18, loss = 79.94574724
Iteration 19, loss = 72.68576188
Iteration 20, loss = 66.17142442
Iteration 21, loss = 61.09206829
Iteration 22, loss = 56.86078095
Iteration 23, loss = 52.48403984
Iteration 24, loss = 48.52653648
Iteration 25, loss = 45.09936095
Iteration 26, loss = 41.96251747
Iteration 27, loss = 38.95139420
Iteration 28, loss = 36.22022284
Iteration 29, loss = 33.87337068
Iteration 30, loss = 31.7542

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=300, verbose=True)

In [119]:
regr.fit(X_p, y_p)

Iteration 1, loss = 29065.32217228
Iteration 2, loss = 13611.43165639
Iteration 3, loss = 6307.96675691
Iteration 4, loss = 2967.18782684
Iteration 5, loss = 1182.17284455
Iteration 6, loss = 432.57502525
Iteration 7, loss = 257.16810740
Iteration 8, loss = 190.75018614
Iteration 9, loss = 140.40339158
Iteration 10, loss = 109.43844617
Iteration 11, loss = 90.71681345
Iteration 12, loss = 77.93291224
Iteration 13, loss = 69.64514239
Iteration 14, loss = 62.54706842
Iteration 15, loss = 56.89507483
Iteration 16, loss = 51.64912719
Iteration 17, loss = 47.54488944
Iteration 18, loss = 44.23811090
Iteration 19, loss = 41.85407203
Iteration 20, loss = 39.67580951
Iteration 21, loss = 38.10533640
Iteration 22, loss = 36.51889163
Iteration 23, loss = 35.38178581
Iteration 24, loss = 34.35117407
Iteration 25, loss = 33.39922241
Iteration 26, loss = 32.85181279
Iteration 27, loss = 32.07761387
Iteration 28, loss = 31.61805354
Iteration 29, loss = 30.94990335
Iteration 30, loss = 30.46206949
It

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=300, verbose=True)

In [120]:
a=regr.predict(X_h)

In [12]:
print(X_c.shape)
print(X_p.shape)

torch.Size([1426, 1948])
torch.Size([2665, 1948])


In [46]:
print(a[1],y_c[1])

[  0.74102696  15.29903956   6.84828006 677.52535638  45.36729455
   5.92421009] [ 52.86000061  27.24999809   7.36999989  11.98999977 158.17999268
   4.94999981]


In [126]:
b=a.reshape(cols_h,rows_h,6)
f2=plt.figure()
plt.imshow(b[:,:,2])
#plt.title()
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()